<a href="https://colab.research.google.com/github/ArmstrongVo/VoTrongLuan.github.io/blob/main/model_VGGL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# import libraries
import keras
import cv2
import tensorflow as tf
from keras.models import Sequential
from glob import glob
from keras.layers import Dense, Dropout, Flatten,Activation
from keras.layers import Conv2D, MaxPooling2D, Conv3D, BatchNormalization
from keras.layers import LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img,img_to_array, array_to_img
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
# access to data
test='/content/drive/MyDrive/DATA FOR AI/PNEUMONIA/TEST'
train='/content/drive/MyDrive/DATA FOR AI/PNEUMONIA/TRAIN'


In [ ]:
# để hệ thống đạt được độ chính xác cao nhất
# tiến hành pre-processing sử dụng kỹ thuật data agumentation
# thiết lập các thông số cho data augmentation
# xoay ảnh
# lật chiều ngang
# lật dọc
# phóng to hoặc thu nhỏ

train_augen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=0.2, 
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range=0.2)

test_augen = ImageDataGenerator(rescale=1./255)

train_set = train_augen.flow_from_directory(train,
                                            target_size=(224, 224),
                                            batch_size=64,
                                            class_mode='categorical')
                                               
test_set = test_augen.flow_from_directory(test,
                                          target_size=(224, 224),
                                          batch_size=64,
                                          class_mode='categorical')

In [11]:
# cấu trúc CNN
model = Sequential()
# sử dụng 16 bộ lọc kích thước 3*3 để tạo tích chập
# trọng số " he-uniform"
# tạo bộ đệm để đầu ra giống đầu vào
# hạn chế mất mát dữu liệu khi tích chập có giá trị âm
# pooling các giá trị lớn nhát trong vùng 2*2 
model.add(Conv2D(32, kernel_size=(3,3), activation='relu',kernel_initializer='he_uniform', padding='same',  input_shape=(224, 224,3)))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(128, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(Conv2D(128, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(256, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(Conv2D(256, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(512, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(Conv2D(512, kernel_size=(3,3),kernel_initializer='he_uniform', padding='same', activation='relu'))
model.add(LeakyReLU(alpha = 0.1))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())  # chuyển đầu ra 2D thành  1D để đưa vào lớp fullconnected
model.add(Activation("relu"))
model.add(Dense(512))
model.add(Dropout(0.4)) # loại bỏ bớt 1 số thành phần để tránh ovefitting_ không tổng quát tốt được quá trình học
model.add(Dense(3))  # fullconnected 3 ngõ ra 
model.add(Activation("softmax"))   # đưa ra xác suất dự đoán cho mỗi lớp

model.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_76 (Conv2D)          (None, 224, 224, 32)      896       
                                                                 
 leaky_re_lu_40 (LeakyReLU)  (None, 224, 224, 32)      0         
                                                                 
 max_pooling2d_40 (MaxPoolin  (None, 112, 112, 32)     0         
 g2D)                                                            
                                                                 
 conv2d_77 (Conv2D)          (None, 112, 112, 64)      18496     
                                                                 
 leaky_re_lu_41 (LeakyReLU)  (None, 112, 112, 64)      0         
                                                                 
 max_pooling2d_41 (MaxPoolin  (None, 56, 56, 64)       0         
 g2D)                                                 

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

training_model = model.fit(train_set,
                    steps_per_epoch=85,
                    epochs=10,
                    validation_data=test_set,
                    validation_steps=6)
model.save("/content/drive/MyDrive/DATA FOR AI/PNEUMONIA/pneumonia.h5")